In [9]:
from dask.distributed import Client

client = Client(n_workers=2, threads_per_worker=1, memory_limit='1GB')


c:\Users\andru\anaconda3\lib\site-packages\distributed\node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 63435 instead
  warnings.warn(


In [2]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 2,Total memory: 1.86 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54407,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 2
Started: Just now,Total memory: 1.86 GiB
Comm: tcp://127.0.0.1:54422,Total threads: 1
Dashboard: http://127.0.0.1:54423/status,Memory: 0.93 GiB
Nanny: tcp://127.0.0.1:54411,


In [10]:
from dask_ml.datasets import make_classification

X, y = make_classification(n_samples=1000, n_features=20,
                           chunks=100, n_informative=4,
                           random_state=0)

c:\Users\andru\anaconda3\lib\statistics.py:168: RuntimeWarning: overflow encountered in long_scalars
  partials[d] = partials_get(d, 0) + n


In [11]:
X

dask.array<normal, shape=(1000, 20), dtype=float64, chunksize=(100, 20), chunktype=numpy.ndarray>

tornado.application - ERROR - Uncaught exception GET /groups/ws (127.0.0.1)
HTTPServerRequest(protocol='http', host='127.0.0.1:8787', method='GET', uri='/groups/ws', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "C:\Users\andru\AppData\Roaming\Python\Python39\site-packages\tornado\websocket.py", line 942, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
  File "C:\Users\andru\AppData\Roaming\Python\Python39\site-packages\tornado\web.py", line 3208, in wrapper
    return method(self, *args, **kwargs)
  File "c:\Users\andru\anaconda3\lib\site-packages\bokeh\server\views\ws.py", line 149, in open
    raise ProtocolError("Token is expired.")
bokeh.protocol.exceptions.ProtocolError: Token is expired.


In [5]:
y

dask.array<astype, shape=(1000,), dtype=int32, chunksize=(100,), chunktype=numpy.ndarray>

In [6]:
from dask_ml.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [7]:
import dask
import xgboost
import dask_xgboost

C:\Users\andru\AppData\Roaming\Python\Python39\site-packages\dask_xgboost\__init__.py:7: UserWarning: Dask-XGBoost has been deprecated and is no longer maintained. The functionality of this project has been included directly in XGBoost. To use Dask and XGBoost together, please use ``xgboost.dask`` instead https://xgboost.readthedocs.io/en/latest/tutorials/dask.html.
  warnings.warn(


In [8]:
params = {'objective': 'binary:logistic',
          'max_depth': 4}

bst = dask_xgboost.train(client, params, X_train, y_train, num_boost_round=2)

Exception in thread Thread-4:
Traceback (most recent call last):
  File "c:\Users\andru\anaconda3\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "c:\Users\andru\anaconda3\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\andru\AppData\Roaming\Python\Python39\site-packages\dask_xgboost\tracker.py", line 365, in join
    while self.thread.isAlive():
AttributeError: 'Thread' object has no attribute 'isAlive'
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
c:\Users\andru\anaconda3\lib\statistics.py:168: RuntimeWarning: overflow encountered in long_scalars
  partials[d] = partials_get(d, 0) + n
c:\Users\andru\anaconda3\lib\statistics.py:168: RuntimeWarning: overflow encountered in long_scalars
  partials[d] = partials_get(d, 0) + n


KeyboardInterrupt: 

In [ ]:
bst

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

ax = xgboost.plot_importance(bst, height=0.8, max_num_features=9)
ax.grid(False, axis="y")
ax.set_title('Estimated feature importance')
plt.show()

NameError: name 'bst' is not defined

In [ ]:
y_hat = dask_xgboost.predict(client, bst, X_test).persist()
y_hat

In [ ]:
from sklearn.metrics import roc_curve

y_test, y_hat = dask.compute(y_test, y_hat)
fpr, tpr, _ = roc_curve(y_test, y_hat)

In [ ]:
from sklearn.metrics import auc

fig, ax = plt.subplots(figsize=(5, 5))
ax.plot(fpr, tpr, lw=3,
        label='ROC Curve (area = {:.2f})'.format(auc(fpr, tpr)))
ax.plot([0, 1], [0, 1], 'k--', lw=2)
ax.set(
    xlim=(0, 1),
    ylim=(0, 1),
    title="ROC Curve",
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
)
ax.legend();
plt.show()

- A blogpost on dask-xgboost http://matthewrocklin.com/blog/work/2017/03/28/dask-xgboost

- XGBoost documentation: https://xgboost.readthedocs.io/en/latest/python/python_intro.html

- Dask-XGBoost documentation: http://ml.dask.org/xgboost.html